In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from datetime import datetime, date
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from math import sqrt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [18]:
FILES_DIR = './files/'
total = pd.read_csv(FILES_DIR + 'total.csv')

In [ ]:
male = total[total['gender'] == 'M'] # 남자고객데이터
female = total[total['gender'] == 'F'] # 여자고객데이터
android = total[total['age'] == 20] # 20대
ios = total[total['age'] == 30] # 30대

In [3]:
group_data = pd.DataFrame({'count': total.groupby(['session_id', 'user_no', 'item_no', 'gender', 'platform']).size()})
group_data = group_data.reset_index()

group_data2 = total.groupby('session_id')['rating'].mean().reset_index()

group_data3 = pd.merge(group_data, group_data2, on='session_id', how='inner')

group_data3 = group_data3

group_data = group_data3.sort_values(by='rating', ascending=False)

# group_data = group_data.drop_duplicates(subset='session_id', keep='first')

group_data = group_data[group_data['count'] == 1]

group_data

,session_id,user_no,item_no,gender,platform,count,rating
0,00047f91f7624d57ae28662b751624db,hoEZFZeX2JUz9+RmAgvubQ==,iP6hnNonEYCWvtV/wsjilA==,F,ANDROID,1,4.0
12100,bba908b4a15aec7237d3f81e93af4eec,X2EtZNswNUi0NQ811Lk0ZA==,Gx7FQgogekHlIMM3ELkHtQ==,F,IOS,1,4.0
15410,ed936abf233af281ff9a6a93152fd9c6,1PZIaHU/KKbfd+DlVyj0RA==,0whP9oGO0f1EHkVqP4gxfg==,F,IOS,1,4.0
1498,179804d87499ad125855bda4a5e9309e,Q7Mc9PJSqgCI5ozZOsc3HA==,6nccKk24PQkrfPhw7cTsOQ==,M,IOS,1,4.0
12139,bc354cf7b5f838d2b1e8a51a0282acdd,/A0eX40c7U/3uiLG/akkTA==,18N28EpFbjcDiUeef3TPug==,F,ANDROID,1,4.0
...,...,...,...,...,...,...,...
6126,5fbfa5744a5b91041a6eeb4ecc15111b,laoJc6XwqswFhQ+6BB0p2A==,MWgv/9pRTxstSyMM2dF9dA==,F,IOS,1,1.0
6127,5fc33d9111e1105614bd8edbdf3f18b2,ccQ8/m6yuvjZKiWD+/KHww==,KGADz14c9pHc+BuwReOQBw==,F,IOS,1,1.0
6128,5fc39b28f229c9b49342e5befce306c9,j1y/B/jOUQd0mb2vyQIvBQ==,y8glepMabn1ukjk43CJWCg==,F,IOS,1,1.0
6129,5fc4f901e508009d863fc365fac5f061,n6dD9L7YfVfC+vxvIk1eMQ==,up5cIpRinQ618fkewQzbWg==,F,IOS,1,1.0


In [19]:
total.columns

Index(['Unnamed: 0', 'session_id', 'event_timestamp', 'event_name', 'user_no',
       'item_no', 'device_type', 'region', 'platform', 'rating', 'birth_date',
       'gender', 'age', 'item_name', 'image_name', 'price', 'category1_code',
       'category1_name', 'category2_code', 'category2_name', 'category3_code',
       'category3_name', 'brand_no', 'brand_name'],
      dtype='object')

In [20]:
def _trans_date_time(df):
    df.index = pd.to_datetime(df['event_timestamp'], infer_datetime_format=True)
    df.drop('event_timestamp', axis=1, inplace=True)
    df.drop('Unnamed: 0', axis=1, inplace=True)
    return df

_trans_date_time(total)

,session_id,event_name,user_no,item_no,device_type,region,platform,rating,birth_date,gender,...,image_name,price,category1_code,category1_name,category2_code,category2_name,category3_code,category3_name,brand_no,brand_name
event_timestamp,,,,,,,,,,,,,,,,,,,,,
2021-06-11 00:36:40.265024,c8c66cdfc2286951af36bd11f26575f2,click_item,5cDoQ298+z7/vx/fNVoZFw==,NdCPDxLau3caXTbPoO2KFA==,mobile,Seoul,IOS,1,1976-05-10,F,...,0e8ff9cc6221b8c4c060786c1c2e0bc5,198000.0,243100100,의류,243103100,원피스/점프수트,243103101,원피스,3739.0,avamolli
2021-06-29 11:47:03.741037,35101c1b373142946d4ab22787a809b9,click_item,moLuwbMr1ZLKJoBHmRgCqg==,9viu9/Vp1NMnPXgmvxKdrw==,mobile,Seoul,ANDROID,1,1961-03-20,F,...,a6997ae34edb96e42ffbbccdd1a1b00a,58000.0,243100100,의류,243104100,하의,243104101,바지,13865.0,ucberkeley
2021-07-11 22:31:42.904002,5c901cfba8cf2c4d25b638aef513a52a,click_item,QA9tQMeKUyUIgAKlbSfpYw==,1B4CTtvF1BeNFZzrdtx4vQ==,mobile,Gangwon-do,IOS,1,1987-09-23,M,...,dc38dced60dfa3a5ac19d367bb10a3c5,67500.0,243100100,의류,243104100,하의,243104101,바지,2649.0,THOMASMORE
2021-08-02 19:09:15.173024,58ac3e65ccf2eb4988b58208f252d359,click_item,bCMrhzWWzYxzqInKRUFzew==,ZnZNKHuqlz863Bo1wkKcLw==,mobile,Daegu,IOS,1,2000-03-28,F,...,ada54816bc84fde22196b7df843ed754,39900.0,244100100,잡화,244106100,주얼리,244106103,목걸이,11744.0,CLUE
2021-07-18 16:15:52.274029,990ecc138c1cbd618aca7cfd08cd1aa8,click_item,/B+O3jb7d/LeH4nKOZDpKg==,11ZJqzJf0OKHy5xVWscPrw==,mobile,Gyeongsangnam-do,IOS,1,1982-04-16,F,...,913e07943fe547acfdd1be8939c10e9b,59000.0,248100100,가구/인테리어,248112100,커튼/블라인드,248112102,커튼/로만세트,7726.0,에이미루시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-15 18:54:19.918064,010a10c3d038a9778b054616988a3840,click_item,OKOXQiBMaEet0w4tVwlrCw==,bMgc9d9iMOXMI1H+JkdHYw==,mobile,Seoul,IOS,1,1994-01-22,M,...,dedbd76d8592b0e91b1786b8f329c106,49900.0,243100100,의류,243102100,상의,243102103,티셔츠,13304.0,polham
2021-06-23 18:10:56.236026,bfdb2e825c648d3153a5709492f925a0,click_item,DYhM5p/b/7nkhUZCJvXj5A==,z10Ej/yvd1gRFxX6rl0XKQ==,mobile,Seoul,IOS,1,1991-03-14,F,...,5dec0844e00c4b5266d97d331d972e85,33220.0,243100100,의류,243104100,하의,243104101,바지,6632.0,toii
2021-07-31 12:40:03.685106,e7cee6d0db15ce5cf52189c0053a93e8,click_item,9j/RJVy3uuEiJ7NBZlZAYQ==,OWkbrvUMsJWYui+puYQhog==,mobile,Seoul,IOS,1,1998-12-16,F,...,2914aa53f6b0a5a5f57975a56b95f89d,85000.0,243100100,의류,243103100,원피스/점프수트,243103101,원피스,6513.0,LOEIL


In [25]:
object_df = total.select_dtypes("object")
int_df = total.select_dtypes("int")
float_df = total.select_dtypes("float")

In [37]:
int_df

,rating,age,category1_code,category2_code,category3_code
event_timestamp,,,,,
2021-06-11 00:36:40.265024,1,40,243100100,243103100,243103101
2021-06-29 11:47:03.741037,1,60,243100100,243104100,243104101
2021-07-11 22:31:42.904002,1,30,243100100,243104100,243104101
2021-08-02 19:09:15.173024,1,20,244100100,244106100,244106103
2021-07-18 16:15:52.274029,1,40,248100100,248112100,248112102
...,...,...,...,...,...
2021-07-15 18:54:19.918064,1,20,243100100,243102100,243102103
2021-06-23 18:10:56.236026,1,30,243100100,243104100,243104101
2021-07-31 12:40:03.685106,1,20,243100100,243103100,243103101


In [36]:
float_df = float_df.reset_index()
float_df

,event_timestamp,price,brand_no
0,2021-06-11 00:36:40.265024,198000.0,3739.0
1,2021-06-29 11:47:03.741037,58000.0,13865.0
2,2021-07-11 22:31:42.904002,67500.0,2649.0
3,2021-08-02 19:09:15.173024,39900.0,11744.0
4,2021-07-18 16:15:52.274029,59000.0,7726.0
...,...,...,...
16623,2021-07-15 18:54:19.918064,49900.0,13304.0
16624,2021-06-23 18:10:56.236026,33220.0,6632.0
16625,2021-07-31 12:40:03.685106,85000.0,6513.0
16626,2021-06-17 23:39:19.132287,42000.0,4630.0


In [30]:
le = LabelEncoder()
total_df = pd.DataFrame(columns=object_df.columns)

for i in object_df.columns:
    le.fit(object_df[i])
    le_incoder = le.transform(object_df[i])
    total_df[i] = le_incoder

In [40]:
int_df_2 = int_df.add_suffix('_int')
float_df_2 = float_df.add_suffix('_float')

In [41]:
int_df_2

,rating_int,age_int,category1_code_int,category2_code_int,category3_code_int
event_timestamp,,,,,
2021-06-11 00:36:40.265024,1,40,243100100,243103100,243103101
2021-06-29 11:47:03.741037,1,60,243100100,243104100,243104101
2021-07-11 22:31:42.904002,1,30,243100100,243104100,243104101
2021-08-02 19:09:15.173024,1,20,244100100,244106100,244106103
2021-07-18 16:15:52.274029,1,40,248100100,248112100,248112102
...,...,...,...,...,...
2021-07-15 18:54:19.918064,1,20,243100100,243102100,243102103
2021-06-23 18:10:56.236026,1,30,243100100,243104100,243104101
2021-07-31 12:40:03.685106,1,20,243100100,243103100,243103101


In [38]:
total_df = pd.concat([total_df, int_df], axis=1)
total_df = pd.concat([total_df, float_df], axis=1)
total_df

C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11560\3586812531.py:1: RuntimeWarning: '<' not supported between instances of 'int' and 'Timestamp', sort order is undefined for incomparable objects.
  total_df = pd.concat([total_df, int_df], axis=1)


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [31]:
total_df

,session_id,event_name,user_no,item_no,device_type,region,platform,birth_date,gender,item_name,image_name,category1_name,category2_name,category3_name,brand_name
0,12733,1,1676,5170,0,28,1,849,0,7457,712,7,49,226,1463
1,3326,1,11116,2379,0,28,0,69,0,1198,8166,7,74,104,1840
2,5831,1,6255,648,0,9,1,3339,1,9020,10814,7,74,104,1280
3,5579,1,8652,7533,0,7,1,7086,0,12066,8517,8,60,91,279
4,9649,1,270,618,0,14,1,1957,0,9979,7068,0,68,313,2135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16623,61,1,5821,7842,0,28,1,5277,1,9362,10936,7,25,344,1744
16624,12175,1,3409,12414,0,28,1,4352,0,11508,4590,7,74,104,1828
16625,14777,1,2540,5372,0,28,1,6776,0,3882,2032,7,49,226,750
16626,695,1,4504,8017,0,12,1,5017,0,4021,5272,7,25,344,1345


In [29]:
le = LabelEncoder()
total_df = pd.DataFrame()
for i in object_df.columns:
    le.fit(object_df[i])
    le_incoder = le.transform(object_df[i])
    total_df.append(le_incoder)

C:\Users\bongkyun\AppData\Local\Temp\ipykernel_11560\1967460946.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df.append(le_incoder)


TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

In [24]:
le = LabelEncoder()
le.fit(object_df)

total_label = le.transform(object_df)
total_label

ValueError: y should be a 1d array, got an array of shape (16628, 15) instead.

In [3]:
# train, test split
x = total.copy()
y = total['session_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)


def RMSE(y_true, y_pred):
  return np.sqrt(np.mean(np.power(np.array(y_true)-np.array(y_pred), 2)))

def score(model):
  id_pairs = zip(x_test['session_id'], x_test['item_no'])
  y_pred = np.array([model(user, item) for (user, item) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)



In [4]:
rating_matrix = total.pivot(index='session_id', columns='item_no', values='rating')
# 아이템 기반 유사도 측정
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)
item_similarity = pd.DataFrame(cosine_similarity(matrix_dummy), index=rating_matrix_t.index, columns=rating_matrix_t.index)

def CF_IBCF(session_id, item_no):
  if item_no in item_similarity:
    sim_scores = item_similarity[item_no]
    user_rating = rating_matrix_t[session_id]
    non_rating_idx = user_rating[user_rating.isnull()].index
    user_rating = user_rating.dropna()
    sim_scores = sim_scores.drop(non_rating_idx)
    mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
  else:
    mean_rating = 2.5
  
  return mean_rating
  
print('아이템 기반 협업 필터링 방식의 정확도:', score(CF_IBCF))

ValueError: Index contains duplicate entries, cannot reshape